In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from pydantic import BaseModel, Field
load_dotenv()
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
os.environ["REQUESTS_CA_BUNDLE"] = r"../../ca-bundle-full.crt"

In [3]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
) 

In [46]:
#llm.invoke("Can you scrape a website")

In [23]:
class Person(BaseModel):
    name: str = Field(description="Name of the Executive")
    title: str = Field(description="Designation or title of the executive")

class Executives(BaseModel):
    people: list[Person] = Field(description="List of Executive of the Company")

In [24]:
output_parser = PydanticOutputParser(pydantic_object = Executives)
format_instructions = output_parser.get_format_instructions()

In [35]:
#format_instructions

In [40]:
human_text = "Get all the executive names and designation of {instruction}\n{format_instructions}"
message = HumanMessagePromptTemplate.from_template(human_text)
prompt = ChatPromptTemplate.from_messages([message])

chain = prompt | llm | output_parser

In [45]:
chain.invoke({"instruction":"Apple Inc","format_instructions":format_instructions}).people

[Person(name='Tim Cook', title='CEO'),
 Person(name='Jeff Williams', title='COO'),
 Person(name='Luca Maestri', title='CFO'),
 Person(name='Katherine Adams', title='General Counsel'),
 Person(name="Deirdre O'Brien", title='SVP of Retail + People'),
 Person(name='Craig Federighi', title='SVP of Software Engineering'),
 Person(name='Johny Srouji', title='SVP of Hardware Technologies'),
 Person(name='Dan Riccio', title='SVP of Hardware Engineering'),
 Person(name='Lisa Jackson', title='VP of Environment, Policy and Social Initiatives')]

In [47]:
chain.invoke({"instruction":"Nvidia","format_instructions":format_instructions}).people

[Person(name='Jensen Huang', title='CEO'),
 Person(name='Colette Kress', title='CFO'),
 Person(name='Debora Shoquist', title='EVP, Operations'),
 Person(name='Tony Tamasi', title='SVP, Content and Technology'),
 Person(name='Jeff Fisher', title='SVP, Gaming'),
 Person(name='Chris Malachowsky', title='Co-founder'),
 Person(name='Bill Dally', title='Chief Scientist'),
 Person(name='Aarti Shah', title='Chief Information Officer'),
 Person(name='Tim Teter', title='Senior Director, Investor Relations')]